# problematic and goal

In order to train a CNN, the shape of the input should be equal for all samples.
the width and height are 512x512 for each slice but the number of slices differs.
We are going to create samples with equal number of slices.

# installing the requirements

In [1]:
# %pip install glob2
# %pip install pytest-shutil

In [2]:
from glob import glob # directory exploration
import shutil # managing files
import os # managing paths

# Creating groups of 64 slices

In [3]:
in_path = 'datasets/dicom_files/images' # created by 3dslicer after processing nifti files
out_path = 'datasets/dicom_groups/images'

In [4]:
for patient in glob(in_path + '/*'):
    patient_name = os.path.basename(os.path.normpath(patient))
    number_folders = int(len(glob(patient+'/*'))/64)

    for i in range(number_folders):
        out_path_name = os.path.join(out_path, patient_name+'_'+str(i))
        os.mkdir(out_path_name)
        for i, file in enumerate(glob(patient+'/*')):
            if i == 64:
                break
            shutil.move(file, out_path_name)


# Convert the dicom files into nifties

In [5]:
# %pip install dicom2nifti

In [6]:
import dicom2nifti

In [7]:
in_path_images = 'datasets/dicom_groups/images'
in_path_labels = 'datasets/dicom_groups/labels'

out_path_images = 'datasets/nifti_files/images'
out_path_labels = 'datasets/nifti_files/labels'

In [8]:
list_images = glob(in_path_images)
list_labels = glob(in_path_labels)

In [9]:
for patient in list_images:
    patient_name = os.path.basename(os.path.normpath(patient))
    dicom2nifti.dicom_series_to_nifti(patient, os.path.join(out_path_images, patient_name + '.nii.gz'))

In [10]:
for patient in list_labels:
    patient_name = os.path.basename(os.path.normpath(patient))
    dicom2nifti.dicom_series_to_nifti(patient, os.path.join(out_path_labels, patient_name + '.nii.gz'))

# Eliminate unwanted patients (those their slices does not contain the liver organ)

In [11]:
# %pip install nibabel

In [12]:
import nibabel as nib
import numpy as np

In [13]:
input_nifti_file_path = 'datasets/nifti_files/labels/'
list_labels = glob(input_nifti_file_path + '*')
for patient in list_labels:
    nifti_file = nib.load(patient)
    fdata = nifti_file.get_fdata()
    np_unique = np.unique(fdata)
    if len(np_unique) == 1:
        shutil.rmtree(patient)